In [2]:
import pandas as pd
import json
from IPython.display import display
import os
import geopandas as gpd
from geopandas.testing import assert_geodataframe_equal
from shapely import wkt

# Atrybuty z OSM - złączenie USA i PL

In [6]:
ny = pd.read_csv("../data/downstream_tasks/airbnb/new_york/New York City_all_tags_one_hot_encoded.csv", nrows=2)
ny = ny.drop(["osmid", 'geometry'], axis=1)
ny

,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_holding_position,aeroway_jet_bridge,aeroway_navigationaid,aeroway_parking_position,...,waterway_fuel,waterway_link,waterway_lock_gate,waterway_put_in,waterway_river,waterway_sluice_gate,waterway_stream,waterway_tidal_channel,waterway_waterfall,waterway_weir
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
st = pd.read_csv("../data/downstream_tasks/airbnb/seattle/Seattle_all_tags_one_hot_encoded.csv", nrows=2)
st = st.drop(["osmid", 'geometry'], axis=1)
st

,feature_id,aeroway_aerodrome,aeroway_apron,aeroway_hangar,aeroway_helipad,aeroway_holding_position,aeroway_navigationaid,aeroway_parking_position,aeroway_runway,aeroway_service,...,waterway_ditch,waterway_dock,waterway_drain,waterway_fish_pass,waterway_lock_gate,waterway_river,waterway_slough,waterway_stream,waterway_waterfall,waterway_yes
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
nycols = list(ny.columns)

In [9]:
len(nycols + list(st.columns))

1885

In [10]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
all_cols = []
for city in cities_pol:
    df = pd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    df = df.drop(["region_id", "geometry"], axis=1)
    city_cols = df.columns
    all_cols.extend(city_cols)

In [11]:
all_cols.extend(nycols)
all_cols.extend(list(st.columns))

In [12]:
len(all_cols)

6063

In [13]:
len(set(all_cols))

2152

In [14]:
all_attrs = {}
all_attrs["cols"] = list(set(all_cols))
with open('../data/hex_attributes.json', 'w') as f:
    json.dump(all_attrs, f)

In [15]:
with open('../data/hex_attributes.json', 'r') as file:
        all_attrs = json.load(file)

In [45]:
all_columns = all_attrs['cols']
all_columns = [element for element in all_columns if element != "accidents_count"]

In [46]:
all_columns_ordered = all_columns.copy()
all_columns_ordered.insert(0, "region_id")
all_columns_ordered.insert(1, "geometry")

In [47]:
len(all_columns)

2151

In [48]:
all_columns_ordered

['region_id',
 'geometry',
 'amenity_gym',
 'building_garages',
 'landuse_gravel',
 'sport_ultimate',
 'office_bakery',
 'natural_shrubbery',
 'landuse_gress',
 'building_guard_cabin',
 'leisure_fitness_centre',
 'shop_spa',
 'shop_slot_machine',
 'historic_machinery',
 'building_subway_entrance',
 'sport_basketball',
 'amenity_dive_centre',
 'sport_karate',
 'leisure_martial_arts',
 'historic_bridge',
 'sport_indoor_skiing;climbing',
 'sport_bodybuilding',
 'amenity_social_club',
 'office_university',
 'building_pavilion',
 'building_recreation',
 'office_nursing_service',
 'shop_comic_books',
 'sport_tennis;volleyball',
 'sport_climbing;squash;table_tennis;dancing',
 'office_software',
 'amenity_fire_station',
 'shop_fish',
 'shop_frozen_food',
 'tourism_camp_site',
 'office_engineer',
 'shop_dry_cleaning',
 'office_auctioneer',
 'office_tax_advisor',
 'amenity_alternative_medicine',
 'shop_party',
 'historic_aqueduct',
 'shop_antiquities;coins',
 'historic_biulding',
 'building_chap

# Dodanie kolumn zgodnie z tagami

## Count embedder

In [49]:
cities_pol = ["poznan", "wroclaw", "warszawa", "krakow", "szczecin"]
year=2022
for city in cities_pol:
    gdf = gpd.read_parquet(f"../data/organized-datasets/{city}/2022/h9/count-embedder/dataset.parquet")
    for col in all_columns:
        if col not in gdf.columns:
            gdf[col] = 0
    gdf = gdf[all_columns_ordered]
    display(gdf)
    if not os.path.exists(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/{city}/{year}/h9/count-embedder/dataset.parquet")

    assert_geodataframe_equal(gdf, gdf_test, check_like=True)


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,891e24aa333ffff,"POLYGON ((16.92005 52.37637, 16.92009 52.37472...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e24a338bffff,"POLYGON ((16.95623 52.46905, 16.95627 52.46741...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e24aa667ffff,"POLYGON ((16.86364 52.38452, 16.86368 52.38287...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e24aa063ffff,"POLYGON ((16.89428 52.38548, 16.89432 52.38383...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e24b838fffff,"POLYGON ((16.80072 52.44687, 16.80076 52.44523...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,891e24a803bffff,"POLYGON ((16.91838 52.33916, 16.91842 52.33751...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2941,891e24aa4d7ffff,"POLYGON ((16.86291 52.41420, 16.86295 52.41255...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2942,891e24a85bbffff,"POLYGON ((16.93311 52.36439, 16.93315 52.36274...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2943,891e24ab58bffff,"POLYGON ((16.83312 52.37860, 16.83317 52.37695...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,891e204e4a7ffff,"POLYGON ((17.04297 51.07181, 17.04301 51.07013...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e204239bffff,"POLYGON ((16.93065 51.15931, 16.93069 51.15763...",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e2041837ffff,"POLYGON ((16.97292 51.06466, 16.97295 51.06298...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e2047223ffff,"POLYGON ((17.09217 51.11872, 17.09221 51.11703...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e2046657ffff,"POLYGON ((17.12372 51.17263, 17.12376 51.17095...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,891e2041137ffff,"POLYGON ((16.91990 51.08073, 16.91994 51.07905...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3164,891e2042273ffff,"POLYGON ((16.90335 51.14586, 16.90339 51.14418...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3165,891e20430d7ffff,"POLYGON ((16.84084 51.13635, 16.84088 51.13466...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3166,891e2050a13ffff,"POLYGON ((16.88187 51.20572, 16.88191 51.20404...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,891f53ca5cfffff,"POLYGON ((21.04469 52.34512, 21.04463 52.34346...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891f522611bffff,"POLYGON ((20.93693 52.20629, 20.93687 52.20463...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891f5352e6fffff,"POLYGON ((21.21538 52.17771, 21.21532 52.17605...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891f535046fffff,"POLYGON ((21.16087 52.16057, 21.16080 52.15891...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891f53ca6a3ffff,"POLYGON ((21.01083 52.33283, 21.01077 52.33118...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5570,891f53c160bffff,"POLYGON ((21.07095 52.35991, 21.07089 52.35825...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5571,891f522418fffff,"POLYGON ((20.95831 52.15892, 20.95826 52.15726...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5572,891f5352b43ffff,"POLYGON ((21.25334 52.16504, 21.25327 52.16338...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5573,891f53504afffff,"POLYGON ((21.16904 52.17298, 21.16898 52.17132...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,891e2e698c3ffff,"POLYGON ((20.00258 49.99518, 20.00254 49.99346...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891e2e79abbffff,"POLYGON ((19.88302 50.10507, 19.88299 50.10336...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891e2e693d7ffff,"POLYGON ((19.93560 49.99747, 19.93557 49.99576...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891e2e6b09bffff,"POLYGON ((19.90475 50.07181, 19.90472 50.07010...",0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891e2e79b13ffff,"POLYGON ((19.91032 50.10264, 19.91028 50.10093...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,891e2e68137ffff,"POLYGON ((20.04835 50.04672, 20.04831 50.04501...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3413,891e2e61b77ffff,"POLYGON ((20.16126 50.09584, 20.16122 50.09413...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3414,891e05a64b3ffff,"POLYGON ((19.84877 49.99962, 19.84874 49.99791...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3415,891e2e7994fffff,"POLYGON ((19.92289 50.11039, 19.92286 50.10868...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,891f0e7b2d3ffff,"POLYGON ((14.47465 53.44809, 14.47475 53.44647...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,891f0e78e8fffff,"POLYGON ((14.65338 53.44017, 14.65348 53.43855...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,891f0e79357ffff,"POLYGON ((14.54000 53.39308, 14.54010 53.39147...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,891f0e7937bffff,"POLYGON ((14.54545 53.38849, 14.54554 53.38688...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,891f0e4e937ffff,"POLYGON ((14.46769 53.47689, 14.46779 53.47527...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,891f1db6a6bffff,"POLYGON ((14.53327 53.33432, 14.53336 53.33270...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3530,891f0e796cfffff,"POLYGON ((14.48220 53.40957, 14.48230 53.40796...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3531,891f0e7a9d3ffff,"POLYGON ((14.68903 53.49299, 14.68913 53.49137...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3532,891f1db69b7ffff,"POLYGON ((14.59808 53.37170, 14.59817 53.37009...",0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
from pandas.testing import assert_series_equal


gdf1 = gpd.read_parquet(f"../data/organized-hexes/wroclaw/2022/h9/count-embedder/dataset.parquet")
gdf2 = gpd.read_parquet(f"../data/organized-hexes/szczecin/2022/h9/count-embedder/dataset.parquet")
# matching names, order and dtypes pass the assertion
assert_series_equal(gdf1.dtypes, gdf2.dtypes)

In [51]:
cities_usa = ["New_York"]
year=2024
for city in cities_usa:
    df = pd.read_csv(f"../data/organized-datasets/organized_USA/{city}_res_9_features.csv")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    df_geom = pd.read_csv(f"../data/downstream_tasks/airbnb/new_york/New York City_res_9.csv")
    merged = df.merge(df_geom, on="region_id", how='inner')
    merged = merged[all_columns_ordered]
    display(merged)
    
    merged['geometry'] = merged['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(merged, geometry='geometry')
    if not os.path.exists(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/new_york/{year}/h9/count-embedder/dataset.parquet")
    assert_geodataframe_equal(gdf, gdf_test, check_like=True)

/tmp/ipykernel_117161/2597814001.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/2597814001.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/2597814001.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipyke

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,892a1000003ffff,POLYGON ((-73.78198948843054 40.86017921842936...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,892a1000007ffff,POLYGON ((-73.77975448365349 40.85758861169454...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,892a100000bffff,POLYGON ((-73.78627525032026 40.86007112650341...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,892a100000fffff,"POLYGON ((-73.78404000610331 40.857480601484, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,892a1000013ffff,POLYGON ((-73.77993873829112 40.86287788509053...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10752,892a10776cbffff,POLYGON ((-74.02132692596481 40.67672226431196...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10753,892a10776cfffff,POLYGON ((-74.01908705622313 40.67414212649311...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10754,892a10776d3ffff,"POLYGON ((-74.01502796991744 40.679536127653, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10755,892a10776d7ffff,POLYGON ((-74.01278832283833 40.67695578075376...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
gdf3 = gpd.read_parquet(f"../data/organized-hexes/new_york/2024/h9/count-embedder/dataset.parquet")
assert_series_equal(gdf1.dtypes, gdf3.dtypes)

In [53]:
cities_usa = ["Seattle"]
year=2024
for city in cities_usa:
    df = pd.read_csv(f"../data/organized-datasets/organized_USA/{city}_res_9_features.csv")
    for col in all_columns:
        if col not in df.columns:
            df[col] = 0
    df_geom = pd.read_csv(f'../data/downstream_tasks/airbnb/seattle/Seattle_res_9.csv')
    merged = df.merge(df_geom, on="region_id", how='inner')
    merged = merged[all_columns_ordered]
    display(merged)
    merged['geometry'] = merged['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(merged, geometry='geometry')
    if not os.path.exists(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/"):
        os.makedirs(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/")
    gdf.to_parquet(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/dataset.parquet")
    gdf_test = gpd.read_parquet(f"../data/organized-hexes/seattle/{year}/h9/count-embedder/dataset.parquet")
    assert_geodataframe_equal(gdf, gdf_test, check_like=True)

/tmp/ipykernel_117161/2509744818.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/2509744818.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipykernel_117161/2509744818.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
/tmp/ipyke

,region_id,geometry,amenity_gym,building_garages,landuse_gravel,sport_ultimate,office_bakery,natural_shrubbery,landuse_gress,building_guard_cabin,...,shop_eggs,historic_heritage,building_government,aeroway_navigationaid,historic_park,historic_train_station,shop_hobby,building_floating_home,amenity_vacuum_cleaner,building_castle
0,8928d540003ffff,POLYGON ((-122.25110252634171 47.6504267090504...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8928d540007ffff,POLYGON ((-122.2550930308571 47.65151394545429...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8928d54000bffff,POLYGON ((-122.24731559220862 47.6521925423558...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8928d54000fffff,POLYGON ((-122.25130621861749 47.6532798738718...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8928d540013ffff,POLYGON ((-122.2508988476786 47.64757342279738...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4198,8928d5cdbcbffff,POLYGON ((-122.39215187876377 47.4929055470186...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4199,8928d5cdbcfffff,POLYGON ((-122.39613348154163 47.4939917364338...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4200,8928d5cdbd3ffff,POLYGON ((-122.39571739712908 47.4882749082355...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4201,8928d5cdbd7ffff,POLYGON ((-122.39969874427203 47.4893610887909...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
gdf3 = gpd.read_parquet(f"../data/organized-hexes/seattle/2024/h9/count-embedder/dataset.parquet")
assert_series_equal(gdf1.dtypes, gdf3.dtypes)